<a href="https://colab.research.google.com/github/Tseng0318/yolo_waste/blob/main/fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Start with extracting and moving data

In [ ]:
import os

drive_path = "/content/drive/MyDrive/Zero_Waste"

print("\nFiles and Folders in Google Drive:")
for root, dirs, files in os.walk(drive_path):
    for file in files:
        print(f"{os.path.join(root, file)}")


📂 Files and Folders in Google Drive:


In [ ]:
import zipfile
dataset_path = "/content/drive/MyDrive/Zero_Waste"

with zipfile.ZipFile(f"{dataset_path}/zerowaste-f.zip", 'r') as zip_ref:
    zip_ref.extractall(dataset_path)

print('Done extracting')

Done extracting


## Start converting json to txt

In [ ]:
import json

In [ ]:
def convert_coco_to_yolo(json_path, images_folder, labels_folder):
    os.makedirs(labels_folder, exist_ok=True)

    # Load COCO JSON
    with open(json_path, "r") as f:
        data = json.load(f)

    # Create a mapping from image_id to its actual filename (without extension)
    image_id_to_filename = {img["id"]: os.path.splitext(img["file_name"])[0] for img in data["images"]}

    # Process each annotation
    for annotation in data["annotations"]:
        image_id = annotation["image_id"]
        category_id = annotation["category_id"]
        bbox = annotation["bbox"]  # COCO format: [x_min, y_min, width, height]

        # Convert COCO bbox to YOLO format
        x_min, y_min, width, height = bbox
        x_center = (x_min + width / 2) / 1280  # Normalize (assuming width = 1280)
        y_center = (y_min + height / 2) / 720  # Normalize (assuming height = 720)
        width /= 1280
        height /= 720

        # Ensure we get the correct filename
        if image_id in image_id_to_filename:
            correct_filename = image_id_to_filename[image_id]  # Matches the corresponding PNG filename

            label_file = os.path.join(labels_folder, f"{correct_filename}.txt")
            with open(label_file, "a") as f:
                f.write(f"{category_id - 1} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

    print(f"✅ Converted {len(data['annotations'])} annotations to YOLO format with correct image names!")

# Convert labels for train, val, and test while ensuring filenames match
for split in ["train", "val", "test"]:
    convert_coco_to_yolo(f"/content/drive/MyDrive/Zero_Waste/{split}/labels.json",
                         f"/content/drive/MyDrive/Zero_Waste/{split}/data",
                         f"/content/drive/MyDrive/Zero_Waste/{split}/labels")

✅ Converted 18002 annotations to YOLO format with correct image names!
✅ Converted 3687 annotations to YOLO format with correct image names!
✅ Converted 5077 annotations to YOLO format with correct image names!


In [ ]:
# import json
# import os
# from tqdm import tqdm

# # Define dataset path
# dataset_path = "/content/drive/MyDrive/Zero_Waste/ZeroWaste-Dataset"
# splits = ["train", "val", "test"]

# # Updated class mapping based on your dataset
# category_map = {1: 0, 2: 1, 3: 2, 4: 3}  # COCO category_id → YOLO class_id

# # Function to convert COCO JSON to YOLO format
# def convert_coco_to_yolo(json_path, images_folder, labels_folder):
#     os.makedirs(labels_folder, exist_ok=True)  # Create labels folder if not exists

#     # Load COCO JSON file
#     with open(json_path, "r") as f:
#         data = json.load(f)

#     # Convert each annotation
#     for annotation in tqdm(data["annotations"], desc=f"Processing {json_path}"):
#         image_id = annotation.get("image_id")
#         category_id = annotation.get("category_id")
#         bbox = annotation.get("bbox", [])  # COCO format: [x, y, width, height]

#         # Skip invalid bounding boxes
#         if not bbox or category_id not in category_map:
#             continue

#         # Find corresponding image
#         image_info = next((img for img in data["images"] if img["id"] == image_id), None)
#         if not image_info:
#             continue

#         image_width, image_height = image_info["width"], image_info["height"]
#         image_filename = image_info["file_name"].replace(".jpg", ".txt")

#         # Convert bounding box to YOLO format
#         x_center = (bbox[0] + bbox[2] / 2) / image_width
#         y_center = (bbox[1] + bbox[3] / 2) / image_height
#         width = bbox[2] / image_width
#         height = bbox[3] / image_height

#         # Get YOLO class ID
#         class_id = category_map[category_id]

#         # Save YOLO label
#         yolo_label = f"{class_id} {x_center} {y_center} {width} {height}\n"
#         label_path = os.path.join(labels_folder, image_filename)

#         with open(label_path, "a") as f:
#             f.write(yolo_label)

# # Convert JSON labels for train, val, and test splits
# for split in splits:
#     json_path = os.path.join(dataset_path, split, "labels.json")  # Correct filename
#     images_folder = os.path.join(dataset_path, split, "data")
#     labels_folder = os.path.join(dataset_path, split, "labels")  # Save YOLO labels here

#     convert_coco_to_yolo(json_path, images_folder, labels_folder)

# print("✅ All COCO labels converted to YOLO format and saved in 'labels/' folders!")

In [ ]:
# # List first 10 label files inside the labels folder of the train split
# labels_folder = "/content/drive/MyDrive/Zero_Waste/ZeroWaste-Dataset/train/labels"
# print("Generated YOLO labels:", os.listdir(labels_folder)[:10])  # Show first 10 labels

Generated YOLO labels: ['01_frame_001160.PNG', '01_frame_001170.PNG', '01_frame_001180.PNG', '01_frame_001190.PNG', '01_frame_001200.PNG', '01_frame_001210.PNG', '01_frame_001220.PNG', '01_frame_001230.PNG', '01_frame_001240.PNG', '01_frame_001250.PNG']


In [ ]:
# # Define the path to the labels folder
# labels_folder = "/content/drive/MyDrive/Zero_Waste/ZeroWaste-Dataset/train/labels"

# # Rename all label files from .PNG to .txt
# for filename in os.listdir(labels_folder):
#     if filename.endswith(".PNG"):  # Change .PNG to .txt
#         old_path = os.path.join(labels_folder, filename)
#         new_path = os.path.join(labels_folder, filename.replace(".PNG", ".txt"))
#         os.rename(old_path, new_path)

# print("✅ All label files renamed to .txt!")

✅ All label files renamed to .txt!


In [3]:
# from PIL import Image
# from multiprocessing import Pool

# def convert_image(image_path):
#     """ Convert a single PNG image to JPG and delete the PNG file only if it exists. """
#     if image_path.endswith(".PNG") or image_path.endswith(".png"):
#         img = Image.open(image_path).convert("RGB")
#         new_path = image_path.replace(".PNG", ".jpg").replace(".png", ".jpg")
#         img.save(new_path, "JPEG")

#         # Check if the PNG file still exists before deleting
#         if os.path.exists(image_path):
#             os.remove(image_path)
#             return f"✅ Converted & Deleted: {new_path}"
#         else:
#             return f"⚠️ Converted but PNG already deleted: {new_path}"
#     return None

# def convert_images_in_folder(folder_path):
#     """ Convert all PNG images in a folder using multiprocessing. """
#     images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".PNG") or f.endswith(".png")]

#     with Pool(processes=os.cpu_count()) as pool:
#         results = pool.map(convert_image, images)

#     print("\n".join([r for r in results if r]))

# # Convert PNGs in train, val, and test folders
# for split in ["train", "val", "test"]:
#     convert_images_in_folder(f"/content/drive/MyDrive/Zero_Waste/{split}/data")

# print("✅ All images converted to .jpg successfully!")

In [ ]:
for split in ["train", "val", "test"]:
    image_folder = f"/content/drive/MyDrive/Zero_Waste/{split}/data"
    label_folder = f"/content/drive/MyDrive/Zero_Waste/{split}/labels"

    image_files = {f.split('.')[0] for f in os.listdir(image_folder) if f.endswith('.jpg')}
    label_files = {f.split('.')[0] for f in os.listdir(label_folder) if f.endswith('.txt')}

    missing_labels = image_files - label_files
    missing_images = label_files - image_files

    print(f"\n📂 Checking {split}:")
    print(f"✅ {len(image_files)} images found.")
    print(f"✅ {len(label_files)} labels found.")

    if missing_labels:
        print(f"❌ {len(missing_labels)} images without labels! Sample: {list(missing_labels)[:10]}")
    if missing_images:
        print(f"❌ {len(missing_images)} labels without images! Sample: {list(missing_images)[:10]}")


📂 Checking train:
✅ 2947 images found.
✅ 2947 labels found.

📂 Checking val:
✅ 571 images found.
✅ 571 labels found.

📂 Checking test:
✅ 899 images found.
✅ 899 labels found.


In [ ]:
import os
for split in ["train", "val", "test"]:
    image_folder = f"/content/drive/MyDrive/Zero_Waste/{split}/data"
    file_types = set([os.path.splitext(f)[1] for f in os.listdir(image_folder)])

    print(f"\n📂 Checking {split}:")
    print(f"✅ Found file types: {file_types}")

    if ".PNG" in file_types or ".png" in file_types:
        print(f"❌ Warning: Some PNG files were not deleted in {split}!")


📂 Checking train:
✅ Found file types: {'.jpg'}

📂 Checking val:
✅ Found file types: {'.jpg'}

📂 Checking test:
✅ Found file types: {'.jpg'}


In [10]:
yaml_path = "/content/drive/MyDrive/Zero_Waste/dataset.yaml"

if os.path.exists(yaml_path):
    print(f"✅ dataset.yaml exists at: {yaml_path}")
else:
    print(f"❌ dataset.yaml NOT found! Check the path.")
with open(yaml_path, "r") as f:
    print(f.read())

✅ dataset.yaml exists at: /content/drive/MyDrive/Zero_Waste/dataset.yaml

# YOLO dataset configuration file

path: /content/drive/MyDrive/Zero_Waste  # Root dataset directory
train: train/data  # Train images
val: val/data  # Validation images
test: test/data  # Test images

nc: 4  # Number of classes
names: ['rigid_plastic', 'cardboard', 'metal', 'soft_plastic']  # Class names



In [14]:
import random

label_folder = "/content/drive/MyDrive/Zero_Waste/train/labels"
label_files = [f for f in os.listdir(label_folder) if f.endswith(".txt")]

sample_files = random.sample(label_files, min(5, len(label_files)))

for file in sample_files:
    file_path = os.path.join(label_folder, file)
    with open(file_path, "r") as f:
        lines = f.readlines()
        print(f"\n📄 {file}:")
        for line in lines[:5]:  # Show first 5 lines
            print(line.strip())


📄 07_frame_028100.txt:
1 1.387031 1.185694 0.144687 0.406111
1 0.641992 1.214514 0.370547 0.570972
1 0.567227 0.288333 0.270547 0.576667
3 1.449648 0.199792 0.100703 0.248472

📄 07_frame_043300.txt:
0 0.900313 0.172292 0.181875 0.344583
1 0.065977 0.564167 0.131953 0.152222
1 1.305703 0.618611 0.388594 0.351667

📄 09_frame_045400.txt:
0 0.450742 0.759861 0.315234 0.553611
1 0.263594 0.416319 0.156094 0.319306
0 1.008945 0.107847 0.159609 0.215694
3 1.446992 0.388056 0.106016 0.431111
3 1.287031 1.241875 0.166719 0.235694

📄 05_frame_005760.txt:
2 0.074414 0.364375 0.148828 0.283194
3 0.185039 0.659653 0.370078 0.433750
1 0.933203 1.357500 0.255469 0.285000

📄 02_frame_001560.txt:
3 0.843711 1.314097 0.256797 0.370139
3 0.349922 0.292778 0.684063 0.585278
3 0.223203 0.187708 0.446406 0.375417
0 1.389883 0.152361 0.220234 0.241111
1 0.984648 1.321528 0.319922 0.353333


In [19]:
# normalize the labels
def normalize_labels(label_folder, image_folder):
    """ Normalize label values to be between 0 and 1. """
    for label_file in os.listdir(label_folder):
        label_path = os.path.join(label_folder, label_file)
        image_path = os.path.join(image_folder, label_file.replace(".txt", ".jpg"))

        if not os.path.exists(image_path):
            print(f"⚠️ Warning: No matching image for {label_file}, skipping.")
            continue

        with open(label_path, "r") as f:
            lines = f.readlines()

        new_lines = []
        for line in lines:
            parts = line.strip().split()
            class_id = parts[0]
            coords = list(map(float, parts[1:]))

            # If any value is greater than 1, assume incorrect format and rescale
            if any(coord > 1 for coord in coords):
                width, height = 640, 640  # Default YOLO image size
                coords = [coords[0] / width, coords[1] / height, coords[2] / width, coords[3] / height]

            new_lines.append(f"{class_id} {' '.join(map(str, coords))}")

        with open(label_path, "w") as f:
            f.write("\n".join(new_lines))

        # print(f"✅ Normalized: {label_file}")

# Run normalization for all label folders
for split in ["train", "val", "test"]:
    normalize_labels(
        f"/content/drive/MyDrive/Zero_Waste/{split}/labels",
        f"/content/drive/MyDrive/Zero_Waste/{split}/data"
    )

print("✅ All label files have been normalized.")

✅ All label files have been normalized.


In [4]:
yaml_path = "/content/drive/MyDrive/Zero_Waste/dataset.yaml"

with open(yaml_path, "r") as f:
    yaml_content = f.read()
    print("\n📄 dataset.yaml content:\n")
    print(yaml_content)


📄 dataset.yaml content:


# YOLO dataset configuration file

path: /content/drive/MyDrive/Zero_Waste  # Root dataset directory
train: train/images  # Train images
val: val/images  # Validation images
test: test/images  # Test images

nc: 4  # Number of classes
names: ['rigid_plastic', 'cardboard', 'metal', 'soft_plastic']  # Class names



## Train on YOLO

In [5]:
dataset_yaml = """
# YOLO dataset configuration file

path: /content/drive/MyDrive/Zero_Waste  # Root dataset directory
train: train/images  # Train images
val: val/images  # Validation images
test: test/images  # Test images

nc: 4  # Number of classes
names: ['rigid_plastic', 'cardboard', 'metal', 'soft_plastic']  # Class names
"""

# Save the updated YAML file
yaml_path = "/content/drive/MyDrive/Zero_Waste/dataset.yaml"

with open(yaml_path, "w") as f:
    f.write(dataset_yaml)

print(f"✅ Fixed dataset.yaml at: {yaml_path}")

✅ Fixed dataset.yaml at: /content/drive/MyDrive/Zero_Waste/dataset.yaml


In [43]:
import os

for split in ["train", "val", "test"]:
    image_folder = f"/content/drive/MyDrive/Zero_Waste/{split}/images"

    if not os.path.exists(image_folder):
        print(f"❌ Folder missing: {image_folder}")
    else:
        jpg_files = [f for f in os.listdir(image_folder) if f.endswith('.jpg')]
        print(f"✅ Found {len(jpg_files)} .jpg images in {image_folder}")

        if len(jpg_files) == 0:
            print(f"⚠️ No images found in {image_folder}!")

✅ Found 2947 .jpg images in /content/drive/MyDrive/Zero_Waste/train/images
✅ Found 571 .jpg images in /content/drive/MyDrive/Zero_Waste/val/images
✅ Found 899 .jpg images in /content/drive/MyDrive/Zero_Waste/test/images


In [6]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.5/921.5 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [7]:
!rm -rf /content/drive/MyDrive/Zero_Waste/train/data.cache
!rm -rf /content/drive/MyDrive/Zero_Waste/val/data.cache
!rm -rf /content/drive/MyDrive/Zero_Waste/test/data.cache

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov10s.pt")

# Start training
model.train(
    data="/content/drive/MyDrive/Zero_Waste/dataset.yaml",
    epochs=100,
    imgsz=640,
    batch=16,
    device="cuda"  # Use "cpu" if you don't have a GPU
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 15.9M/15.9M [00:00<00:00, 120MB/s] 


Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov10s.pt, data=/content/drive/MyDrive/Zero_Waste/dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show

100%|██████████| 755k/755k [00:00<00:00, 21.2MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1     36096  ultralytics.nn.modules.block.SCDown          [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1    137728  ultralytics

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     27/100      4.84G      3.804      6.394      3.888         19        640: 100%|██████████| 185/185 [01:32<00:00,  2.00it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:09<00:00,  1.93it/s]

                   all        571       3687      0.377     0.0528     0.0363     0.0162



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     28/100      4.86G      3.815      6.309      3.874         83        640:  58%|█████▊    | 108/185 [00:54<00:35,  2.15it/s]